Yesterday (02/09/2024), the group completed the ML models to create the baseline models for both financial agent and sentiment classification. Sasha's notebook, and models are the ones we will use as a baseline. These can be found below:

# Initial Baseline Models

In [1]:
##### INSERT SASHAS BASELINE MODELS #####

# Deep Learning Models

Today, we will try to improve the performance of the baseline models using the following:
- LSTM
- BERT
- FinBert
- Roberta

The first step is to load the data again

## Load in the Data

In [2]:
import pandas as pd
import numpy as np

In [3]:
#!pip install numpy==1.23.4 put in requirments.txt

In [4]:
# Load in the agent classification data
bis_data_aud = pd.read_csv('../raw_data/Labelled/Audiences/BIS_prelabelled.txt', header=0)
ecb_data_aud = pd.read_csv('../raw_data/Labelled/Audiences/ECB_prelabelled.txt', header=0)
fed_data_aud = pd.read_csv('../raw_data/Labelled/Audiences/FED_prelabelled.txt', header=0, engine='python')

# load in the sentiment classifcation data
bis_data_sent = pd.read_csv('../raw_data/Labelled/Sentiment/BIS_prelabelled_sent.txt', header=0)
ecb_data_sent = pd.read_csv('../raw_data/Labelled/Sentiment/ECB_prelabelled_sent.txt', header=0)
fed_data_sent = pd.read_csv('../raw_data/Labelled/Sentiment/FED_prelabelled_sent.txt', header=0)
fed_data_sent = fed_data_sent.drop('audience', axis=1)

In [5]:
# Merge the sentiment classifcation into one df
sentiment_data = pd.concat([fed_data_sent, ecb_data_sent, bis_data_sent])
sentiment_data = sentiment_data.reset_index(drop=True)
sentiment_data.head()

,text,sentiment
0,governmental policy including that of the fede...,0
1,in light of the difficulties and uncertainties...,0
2,this means keeping the federal funds rate at i...,1
3,any decision to move in this direction therefo...,1
4,6the developments in the payments area are par...,0


In [6]:
# Merge the agent classifcation into one df
agent_data = pd.concat([bis_data_aud, fed_data_aud, ecb_data_aud])
agent_data = agent_data.reset_index(drop=True)
agent_data = agent_data.drop(columns=['Unnamed: 0'])
agent_data = agent_data.dropna()
agent_data.head()

,Parsed_Text,label
0,the unemployment rate last year averaged just ...,0.0
1,in addition the end of extended unemployment i...,0.0
2,for persons with only a high school diploma bo...,0.0
3,but once the shortterm unemployed pool is depl...,0.0
4,this is almost the same level as the structura...,0.0


## Long-Short Term Memory (LSTM)

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Masking, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.sequence import pad_sequences

2024-09-03 12:08:08.796764: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-03 12:08:08.800881: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-03 12:08:08.845755: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-03 12:08:08.893396: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-03 12:08:08.927898: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

### Sentiment

In [8]:
X = sentiment_data['text']
y = sentiment_data['sentiment']

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=69, stratify=y)

In [9]:
word2vec = Word2Vec(sentences=X_train, vector_size=60, min_count=10, window=10)

In [10]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

In [11]:
# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

In [12]:
# Embed the training and test sentences
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)


# Pad the training and test embedded sentences
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)

In [13]:
#input_shape = (X_train_pad.shape[1], X_train_pad.shape[2])
from tensorflow.keras import layers
def init_LSTMl():
    model = Sequential()

    # Masking layer to skip padding tokens
    model.add(Masking())

    # LSTM layer
    model.add(layers.LSTM(20, activation='tanh'))

    # Dense hidden layer
    model.add(layers.Dense(15, activation='relu'))

    # Output layer for binary classification
    model.add(layers.Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['precision'])

    return model

LSTM_model = init_LSTMl()

In [14]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

LSTM_model.fit(X_train_pad, y_train, 
          batch_size = 32,
          epochs=100,
          validation_split=0.5,
          callbacks=[es]
         )

Epoch 1/100


2024-09-03 12:08:26.067639: E tensorflow/core/util/util.cc:131] oneDNN supports DT_BOOL only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


148/148 ━━━━━━━━━━━━━━━━━━━━ 47s 293ms/step - loss: 0.6838 - precision: 0.5169 - val_loss: 0.6805 - val_precision: 0.7945
Epoch 2/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 42s 282ms/step - loss: 0.6693 - precision: 0.6774 - val_loss: 0.6803 - val_precision: 0.7188
Epoch 3/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 38s 255ms/step - loss: 0.6716 - precision: 0.6659 - val_loss: 0.6747 - val_precision: 0.5740
Epoch 4/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 36s 245ms/step - loss: 0.6666 - precision: 0.6242 - val_loss: 0.6709 - val_precision: 0.5920
Epoch 5/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 37s 247ms/step - loss: 0.6657 - precision: 0.6039 - val_loss: 0.6733 - val_precision: 0.5378
Epoch 6/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 36s 245ms/step - loss: 0.6618 - precision: 0.6071 - val_loss: 0.6674 - val_precision: 0.5367
Epoch 7/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 37s 249ms/step - loss: 0.6503 - precision: 0.6113 - val_loss: 0.6686 - val_precision: 0.5167
Epoch 8/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 33s 225ms/step - loss: 0.6538 - p

In [15]:
res = LSTM_model.evaluate(X_test_pad, y_test, verbose=0)

print(f'The precision evaluated on the test set is of {res[1]*100:.3f}%')

The precision evaluated on the test set is of 61.982%


## BERT

## FinBert

## Roberta